In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

data_path = '/content/drive/MyDrive/refer-kitti'
    

In [4]:
! rm -rf GMC-Link
! git clone https://github.com/Seanachan/GMC-Link
! cd GMC-Link && pwd
! pip install /content/GMC-Link

Cloning into 'GMC-Link'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 38 (delta 11), reused 32 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (38/38), 17.22 KiB | 2.87 MiB/s, done.
Resolving deltas: 100% (11/11), done.
/content/GMC-Link
Processing ./GMC-Link
  Preparing metadata (setup.py) ... done
  Created wheel for GMC_Link: filename=GMC_Link-0.1-py3-none-any.whl size=13653 sha256=3b28fef5e32f9d54a3e7be09903c0b31b06311f1e69000b26e3cf2928e247850
  Stored in directory: /root/.cache/pip/wheels/dd/8d/60/a850d91def273ef2be84987f282109ec44e6e18d0080262f99
Successfully built GMC_Link


In [5]:
import cv2
import torch
import os
import numpy as np
from gmc_link.manager import GMCLinkManager
from gmc_link.text_utils import TextEncoder

In [6]:
def run_gmc_link_demo(frame_dir, user_prompt):
    # 1. Initialize the system
    device = "cuda" if torch.cuda.is_available() else "cpu"
    encoder = TextEncoder(device=device)
    linker = GMCLinkManager(device=device)
    
    # 2. Encode the Prompt (Done once per video)
    print(f"Encoding prompt: '{user_prompt}'...")
    language_embedding = encoder.encode(user_prompt)

    # 3. Load Frames
    frame_files = sorted([f for f in os.listdir(frame_dir) if f.endswith(('.png', '.jpg'))])
    
    # --- MOCK TRACKING STATE ---
    # In a real RMOT task, these come from your tracker (e.g., ByteTrack)
    track_id = 101
    current_centroid = np.array([500.0, 500.0]) # Starting position

    for frame_name in frame_files:
        frame_path = os.path.join(frame_dir, frame_name)
        frame = cv2.imread(frame_path)
        if frame is None: continue

        # --- SIMULATE MOVEMENT ---
        # Let's simulate an object moving LEFT (-5 pixels per frame)
        prev_centroid = current_centroid.copy()
        current_centroid += np.array([-5.0, 0.0])

        # We wrap this in a simple class to match the manager's expectation
        class Track:
            def __init__(self, tid, pos, prev):
                self.id = tid
                self.centroid = pos
                self.prev_centroid = prev
        
        active_tracks = [Track(track_id, current_centroid, prev_centroid)]

        # --- THE GMC-LINK CORE PROCESS ---
        # 1. Compensates for camera motion
        # 2. Calculates world velocity
        # 3. Aligns with the text embedding
        scores = linker.process_frame(frame, active_tracks, language_embedding)
        
        # --- VISUALIZE ---
        score = scores.get(track_id, 0.0)
        # Green if it matches 'Left', Red if it doesn't
        color = (0, 255, 0) if score > 0.3 else (0, 0, 255)
        
        cv2.circle(frame, tuple(current_centroid.astype(int)), 10, color, -1)
        cv2.putText(frame, f"Score: {score:.2f}", (int(current_centroid[0]), int(current_centroid[1] - 20)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        cv2.putText(frame, f"Prompt: {user_prompt}", (20, 40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        cv2.imshow("GMC-Link RMOT Demo", frame)
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()

In [9]:
if __name__ == "__main__":
    # Point this to your folder of frames!
    MY_FRAMES = f"{data_path}/KITTI/training/image_02/0011"
    # MY_PROMPT = f"{data_path}/expresion/0011/moving-cars.json"
    MY_PROMPT = "moving cars"
    
    run_gmc_link_demo(MY_FRAMES, MY_PROMPT)

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Encoding prompt: 'moving cars'...


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x384 and 768x256)